![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Assertion Status Model 

## Setup

In [2]:
%pip install -q tensorflow==2.7.0
%pip install -q tensorflow-addons

In [4]:
from johnsnowlabs import *

import json
import os

import numpy as np
import pandas as pd

print("Spark NLP Version :", sparknlp.version())

spark = start_spark()

Spark NLP Version : 4.2.1
📋 Loading license number 0 from /home/ubuntu/.johnsnowlabs/licenses/license_number_0_for_.json


22/10/19 11:54:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched cpu-Optimized JVM SparkSession with Jars for: 🚀Spark-NLP==4.2.1, 💊Spark-Healthcare==4.2.0, 🕶Spark-OCR==4.1.0, running on ⚡ PySpark==3.1.2


## Identify COMPETITORS in a Text with Assertion Status

This model uses Assertion Status to identify if a **PRODUCT** or an **ORG** is mentioned to be a competitor. By default, if nothing is mentioned, it returns **NO_COMPETITOR**.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentence_detector =  nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer =  nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings =  nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")\

ner_converter = finance.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\

assertion = finance.AssertionDLModel.pretrained("finassertion_competitors", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter,
    assertion
    ])

empty_df = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_df)

light_model = LightPipeline(model)

### Getting Result 

In [7]:
sample_text = """Our competitors include the following by general category: legacy antivirus product providers, such as McAfee LLC and Broadcom Inc."""

In [8]:
data = spark.createDataFrame([[sample_text]]).toDF("text")

In [9]:
result = model.transform(data)

In [10]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata, result.assertion.result)).alias("cols"))\
      .select(F.expr("cols['1']['sentence']").alias("sent_id"),
              F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label"),
              F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------+------------+---------+----------+
|sent_id|chunk       |ner_label|assertion |
+-------+------------+---------+----------+
|0      |McAfee LLC  |ORG      |COMPETITOR|
|0      |Broadcom Inc|ORG      |COMPETITOR|
+-------+------------+---------+----------+



### Getting Result with LightPipeline

In [11]:
import pandas as pd

light_result = light_model.fullAnnotate(sample_text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

chunks entities   assertion
0    McAfee LLC      ORG  COMPETITOR
1  Broadcom Inc      ORG  COMPETITOR

### Visualization of Assertion Status

In [12]:
# from sparknlp_display import AssertionVisualizer

vis = viz.AssertionVisualizer()

vis.display(light_result, 'ner_chunk', 'assertion')

## Writing a Generic Assertion + NER Function

In [13]:
# from pyspark.sql.functions import monotonically_increasing_id

def get_base_pipeline(embeddings):

    documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

    sentenceDetector = nlp.SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

    tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

    embeddings = nlp.BertEmbeddings.pretrained(embeddings, "en") \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("embeddings")

    base_pipeline = Pipeline(stages=[
                        document_assembler,
                        sentence_detector,
                        tokenizer,
                        embeddings])

    return base_pipeline


def get_assertion (embeddings, ner_model, assertion_model):

    ner = finance.NerModel.pretrained(ner_model, "en", "finance/models")\
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = nlp.NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")
    
    assertion = finance.AssertionDLModel.pretrained(assertion_model, "en", "finance/models")\
        .setInputCols(["sentence", "ner_chunk", "embeddings"])\
        .setOutputCol("assertion")
      
    base_model = get_base_pipeline(embeddings)

    nlpPipeline = Pipeline(stages=[
        base_model,
        ner,
        ner_converter,
        assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)
    
    light_model = LightPipeline(model)
    
    return light_model
    

### Getting Result with LightPipeline

In [14]:
sample_text = """EDH combines our Cloudera Data Warehouse, Cloudera Operational DB, and Cloudera Data Science with our SDX technology."""

embeddings = "bert_embeddings_sec_bert_base"

ner_model = "finner_orgs_prods_alias"

assertion_model = "finassertion_competitors"

light_result = get_assertion(embeddings, ner_model, assertion_model).fullAnnotate(sample_text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

finner_orgs_prods_alias download started this may take some time.
[OK!]
finassertion_competitors download started this may take some time.
[OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]


In [15]:
df

chunks entities      assertion
0                      EDH      ORG  NO_COMPETITOR
1  Cloudera Data Warehouse  PRODUCT  NO_COMPETITOR
2  Cloudera Operational DB  PRODUCT  NO_COMPETITOR
3    Cloudera Data Science  PRODUCT  NO_COMPETITOR
4                      SDX  PRODUCT  NO_COMPETITOR

### Visualization of Assertion Status

In [16]:
vis = viz.AssertionVisualizer()

vis.display(light_result, 'ner_chunk', 'assertion')

## Identify PAST Situations in a Text with Assertion Status

In [17]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

tokenClassifier = finance.BertForTokenClassification.pretrained("finner_bert_roles","en","finance/models")\
    .setInputCols("token", "document")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = finance.NerConverterInternal() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ROLE"])

assertion = finance.AssertionDLModel.pretrained("finassertiondl_past_roles", "en", "finance/models")\
    .setInputCols(["document", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setMaxSentLen(1200)
    
nlpPipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    tokenClassifier,
    ner_converter,
    assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]
finner_bert_roles download started this may take some time.
[ | ]finner_bert_roles download started this may take some time.
Approximate size to download 380.6 MB
[ — ]Download done! Loading the resource.
[OK!]
finassertiondl_past_roles download started this may take some time.
[ | ]finassertiondl_past_roles download started this may take some time.
Approximate size to download 2.1 MB
Download done! Loading the resource.
[OK!]


### Getting Result with LightPipeline

In [21]:
sample_texts = ["""From January 2009 to November 2017, Mr. Tan worked as the Managing Director of Cadence""",
                """Jane S. Smith works as a Computer Engineer and Product Lead at Globalize Cloud Services""",
                """Mrs. Johansson has been apointed CEO and President of Mileways""",
                """Tom Martin worked as Cadence's CTO until 2010""",
                """Mrs. Charles was before Managing Director at a big consultancy company""",
                """We are happy to announce that Mary Leigh joins Elephant as Web Designer and UX/UI Developer"""]

chunks=[]
entities=[]
status=[]

for i in range(len(sample_texts)):
    light_result = light_model.fullAnnotate(sample_texts[i])[0]

    for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
        chunks.append(n.result)
        entities.append(n.metadata['entity']) 
        status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [22]:
df

chunks entities assertion
0           Director     ROLE      PAST
1  Computer Engineer     ROLE   NO_PAST
2       Product Lead     ROLE   NO_PAST
3                CEO     ROLE   NO_PAST
4          President     ROLE   NO_PAST
5      Cadence's CTO     ROLE      PAST
6  Managing Director     ROLE      PAST
7       Web Designer     ROLE   NO_PAST
8    UX/UI Developer     ROLE   NO_PAST

### Visualization of Assertion Status

In [23]:
for i in range(len(sample_texts)):
    
    light_result = light_model.fullAnnotate(sample_texts[i])[0]
    
    vis = viz.AssertionVisualizer()

    vis.display(light_result, 'ner_chunk', 'assertion')

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52536)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ubuntu/.jupyter_env/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/home/ubuntu/.jupyter_env/lib/python3.8/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/home/ubuntu/.jupyter_env/lib/python3.8/site-packages/pyspark/accumulators.py", line 239, in accum_updates
    num_u